In [1]:
import sys

sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tqdm import tqdm

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    FACTOR_MODEL_ESTIMATES,
    FIX_SET_OF_HALF_LIFES,
    SPECIFIC_RISK,
    RETURN_DB,
)
from factor_model.model_update.database_generators.generate_databases import (
    refresh_database,
)
import datetime
from factor_model.risk_calculations.core_universe_portfolio import (
    generate_market_portfolio,
)

EXPOSURE_NON_STYLE_FIELDS = [
    "id",
    "ticker",
    "return",
    "core_universe",
    "transformed_market_cap",
    "date",
]
from factor_model.model_update.database_generators import EXPOSURE_NON_STYLE_FIELDS

from factor_model.risk_calculations.risk_attribution import (
    create_portfolio_exposures,
    generate_factor_covariance_attribution,
    calculate_spec_risk_mctr,
    generate_factor_covariance_table,
    generate_active_space_portfolio,
    get_specific_risk_beta,
)
from factor_model.risk_calculations.factor_covariance import (
    generate_factor_covariance_matrix,
)
from factor_model.risk_calculations.specific_risk import (
    generate_raw_specific_risk,
    generate_raw_portfolio_specific_risk,
)
from factor_model.risk_calculations.risk_metrics import calculate_lognormal_es_var

In [2]:
# A sample portfolio to test calculations and restrict query...
portfolio_details = {
    "BCUBE-USD": 0.17,
    "LNC-USD": 0.05,
    # "PMG24050-USD": 0.2,
    "ZNN-USD": 0.04,
    "EFI-USD": 0.03,
    "BTC-USD": 0.32,
    "ETH-USD": 0.19,
    "USDT-USD": 0.1,
    "BNB-USD": 0.1,
}

market_portfolio = {
    "BTC-USD": 0.31574372070244366,
    "ETH-USD": 0.18711055133973806,
    "USDT-USD": 0.08725061854643708,
    "BNB-USD": 0.06932395681724883,
    "SOL-USD": 0.06602083999170749,
    "STETH-USD": 0.0534749145676961,
    "XRP-USD": 0.05020181639431607,
    "USDC-USD": 0.046874562996328084,
    "ADA-USD": 0.04406750903164565,
    "DOGE-USD": 0.041257706928259585,
    "SHIB-USD": 0.03867380268417949,
}

# risk calculation
risk_calculation_parameters = {
    "correlation_half_life": 730,  # days
    "variance_half_life": 365,  # days
    "specific_risk_half_life": 365,
    "date": "2023-03-01",
    "minimum_history_spec_ret": 730,
}

# portfolio_details= {'BTC-USD': 0.5, 'ETH-USD': 0.5}
# market_portfolio={'BTC-USD': 0.5, 'ETH-USD': 0.5}

# risk calculation
risk_calculation_parameters = {
    "correlation_half_life": 300,  # days
    "variance_half_life": 300,  # days
    "specific_risk_half_life": 300,
    "date": "2023-03-01",
    "minimum_history_spec_ret": 300,
}
# alias
cob_date = risk_calculation_parameters["date"]

#### 0. Load relevant input data

##### Ideally this should be supplied through Django db queries


In [3]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    exposures = pd.read_sql_query(
        f"SELECT * FROM exposures where date = '{cob_date}'",
        conn,
    )

if market_portfolio is None:
    market_portfolio = generate_market_portfolio(exposures)
all_tickers = list(set(market_portfolio.keys()).union(set(portfolio_details.keys())))
all_tickers_str = "','".join(all_tickers)

In [4]:
# factor return to fill miss
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    fill_miss_returns = pd.read_sql_query(
        "SELECT * FROM factor_returns where date <= '{cob_date}'", conn
    )[["date", "market"]].rename(columns={"market": "proxy_return"})
fill_miss_returns["date"] = fill_miss_returns["date"].astype(str)
fill_miss_returns.drop_duplicates(inplace=True)

In [5]:
all_tickers = list(set(market_portfolio.keys()).union(set(portfolio_details.keys())))
all_tickers_str = "','".join(all_tickers)

with sqlite3.connect(os.path.join(DATABASE_LOCATION, RETURN_DB)) as conn:
    return_df = pd.read_sql_query(
        f"SELECT date, return, symbol FROM returns where date <= '{cob_date}' and symbol in ('{all_tickers_str}')",
        conn,
    )
return_df.rename(columns={"return": "excess_return"}, inplace=True)

In [10]:
# force the dataframe into a factor return like format and fill miss the missing values
def generate_processed_excess_returns(
    return_df: pd.DataFrame, fill_miss_returns: pd.DataFrame
) -> pd.DataFrame:
    """Generates the excess return history for the non factor model

    Args:
        return_df (pd.DataFrame): _description_
        fill_miss_returns (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    symbols = set(return_df["symbol"])
    factor_return_formatted_df = None

    for symbol in symbols:
        if factor_return_formatted_df is None:
            factor_return_formatted_df = return_df[return_df["symbol"] == symbol][
                ["date", "excess_return"]
            ].rename(columns={"excess_return": symbol})

        else:
            temp_df = return_df[return_df["symbol"] == symbol][
                ["date", "excess_return"]
            ].rename(columns={"excess_return": symbol})
            factor_return_formatted_df = factor_return_formatted_df.merge(
                temp_df, how="outer", on="date"
            )

    factor_return_formatted_df.sort_values(by="date", inplace=True)
    factor_return_formatted_df["date"] = factor_return_formatted_df["date"].astype(str)
    fill_miss_returns["date"] = fill_miss_returns["date"].astype(str)

    # deploy the fill miss procedure and filter to relevant time horizons

    factor_return_formatted_df = factor_return_formatted_df.merge(
        fill_miss_returns, how="left", on="date"
    )

    first_market_return_date = min(fill_miss_returns["date"])
    factor_return_formatted_df = factor_return_formatted_df[
        factor_return_formatted_df["date"] >= first_market_return_date
    ]

    for symbol in symbols:
        factor_return_formatted_df[symbol] = np.where(
            factor_return_formatted_df[symbol].isnull(),
            factor_return_formatted_df["proxy_return"],
            factor_return_formatted_df[symbol],
        )

    factor_return_formatted_df.drop(columns=["proxy_return"], inplace=True)
    return factor_return_formatted_df

factor_return_formatted_df = generate_processed_excess_returns(return_df, fill_miss_returns)

In [7]:
active_space_port = generate_active_space_portfolio(portfolio_details, market_portfolio)

In [8]:
portfolios = {
    "portfolio": portfolio_details,
    "market": market_portfolio,
    "active": active_space_port,
}
covariance_matrixes = {}
relevant_keys = {}
port_exposures = {}
total_risks = {}
total_attributions = {}
mctrs = {}
factor_covars = {}
for portfolio in portfolios:
    # 1. exposure calc
    port_exposures[portfolio] = pd.Series(portfolios[portfolio]).sort_values(
        ascending=False
    )
    relevant_keys[portfolio] = list(port_exposures[portfolio].index)
    covariance_matrixes[portfolio] = generate_factor_covariance_matrix(
        factor_return_formatted_df[["date"] + relevant_keys[portfolio]],
        risk_calculation_parameters,
    )

    # 2. risk calculation without factors
    (
        total_risks[portfolio],
        total_attributions[portfolio],
    ) = generate_factor_covariance_attribution(
        port_exposures[portfolio].to_frame("exposure"), covariance_matrixes[portfolio]
    )
    mctrs[portfolio] = total_attributions[portfolio] / total_risks[portfolio]

    factor_covars[portfolio] = generate_factor_covariance_table(
        port_exposures[portfolio].to_frame("exposure"), covariance_matrixes[portfolio]
    )

all_exposure = pd.concat([port_exposures["portfolio"], port_exposures["market"]], axis=1).fillna(0)
cov_for_beta = generate_factor_covariance_matrix(
        factor_return_formatted_df[["date"] + list(all_exposure.index)],
        risk_calculation_parameters,
    )


factor_beta_covar, _ = generate_factor_covariance_attribution(
    all_exposure[[0]].rename(columns= {0: "exposure"}),
    cov_for_beta,
    all_exposure[[1]].rename(columns= {1: "exposure"})
)

portfolio_beta = (factor_beta_covar**2) / (total_risks["market"] ** 2)

es95, var95 = calculate_lognormal_es_var(total_risks["portfolio"], 0.95)
es99, var99 = calculate_lognormal_es_var(total_risks["portfolio"], 0.99)

In [9]:
portfolio_beta

0.9435322441134444